In [ ]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
to_run = True
if to_run:
    os.chdir('..')
    to_run = False
print(os.getcwd())

# Option 1 & 2: example input and output

In [1]:
from pm4py.objects.dcr.hierarchical.obj import HierarchicalDcrGraph
graph = HierarchicalDcrGraph()

# Atomic events
events = [
    "A", "B", "C", "D", "E", "F", "G"
]

# Super events
groups = [    
    "N1", "N2", "N3", "N4"
]

for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    graph.marking.included.add(event)

for event in groups:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event

# Set up nested groups
graph.nestedgroups["N1"] =  {"B", "N2"}
graph.nestedgroups["N2"] =  {"C", "D"}
graph.nestedgroups["N3"] =  {"E", "F"}
graph.nestedgroups["N4"] =  {"G"}
for group, events in graph.nestedgroups.items():
    for e in events:
        graph.nestedgroups_map[e] = group

# Set up graph
graph.marking.included.remove("B")
graph.marking.included.remove("G")
graph.marking.pending.add("D")
graph.marking.pending.add("G")

graph.conditions["N1"] = {"A"}
graph.conditions["N3"] = {"N1"}

graph.responses["N2"] = {"N3"}

graph.includes["N3"] = {"B"}
graph.includes["F"] = {"N4"}

graph.excludes["C"] = {"N2"}

graph.milestones["A"] = {"N4"}

########### Testing the semantics ###########
from pm4py.objects.dcr.hierarchical.semantics import HierarchicalDcrSemantics

semantics = HierarchicalDcrSemantics()

errors = 0

expected_enabled = ["A"]
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                print(f"{e} should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1
                print(f"{e} should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1
        print("Graph should NOT be accepting")        


semantics.execute(graph, "A")
expected_enabled = ["A", "C", "D"]
for e in events:
    if e in expected_enabled:
        if not semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should NOT be enabled")
if semantics.is_accepting(graph):
    errors+=1
    print("Graph should NOT be accepting")

semantics.execute(graph, "C")
expected_enabled = ["A", "E", "F"]
for e in events:
    if e in expected_enabled:
        if not semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should NOT be enabled")
if semantics.is_accepting(graph):
    errors+=1 
    print("Graph should NOT be accepting")

semantics.execute(graph, "E")
expected_enabled = ["A", "B"]
for e in events:
    if e in expected_enabled:
        if not semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should NOT be enabled")
if semantics.is_accepting(graph):
    errors+=1
    print("Graph should NOT be accepting")          

semantics.execute(graph, "B")
expected_enabled = ["A", "B", "E", "F"]
for e in events:
    if e in expected_enabled:
        if not semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should NOT be enabled")
if semantics.is_accepting(graph):
    errors+=1
    print("Graph should NOT be accepting")                

semantics.execute(graph, "F")
expected_enabled = ["B", "E", "F", "G"]
for e in events:
    if e in expected_enabled:
        if not semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should NOT be enabled")
if semantics.is_accepting(graph):
    errors+=1
    print("Graph should NOT be accepting")                        

semantics.execute(graph, "G")
expected_enabled = ["A", "B", "E", "F", "G"]
for e in events:
    if e in expected_enabled:
        if not semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
            errors+=1
            print(f"{e} should NOT be enabled")
if not semantics.is_accepting(graph):
    errors+=1
    print("Graph should be accepting")                        


print(f"{errors} errors occured.") # This should be 0


0 errors occured.
